# Measuring Model Uncertainty
Uncertainty in machine learning stems from the inherent unpredictability of real-world data and the limitations of models in capturing all its complexities. Effectively understanding and managing this uncertainty is crucial for successful machine learning applications. In this notebook, we will assess the uncertainty of the OriginalSizeDropoutCNN model using two techniques: ***Monte Carlo Dropout*** and an ***Ensemble of models***. We will then compare the results from both methods to evaluate their effectiveness in quantifying model uncertainty. The comparision of results for both models can be seen here [w&b report](https://wandb.ai/mrpralinka-warsaw-university-of-technology/iml/reports/DropoutCNN-vs-OriginalSizeCNN--VmlldzoxMDQyODgxOQ?accessToken=btmnpz9qmytiqutkcl9mv8xn770mwmm9gp4jbimemh99czvxgnp3mz4b0hzbs4cj)

# Set up imports

In [ ]:
import os
if not os.path.exists("./notebooks"):
    %cd ..

import numpy as np
from src.data_processing import load_mean_std

import torch
from torchvision import transforms
import src.model
from src.training import monte_carlo_predictions, model_validate


# 0. Set Device 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Monte Carlo Dropout
Monte Carlo (MC) Dropout is a technique used to estimate model uncertainty by applying dropout during both training and inference. By performing multiple stochastic forward passes with different neurons dropped each time, we can approximate a distribution over the model's predictions, revealing its uncertainty. To achieve this we will process the sama data through the model with dropout turned on.

# 1. Define Monte Carlo Dropout testing


In [ ]:
def monte_carlo_dropout(model, test_loader, samples = 20):
    predictions = []
    for _ in range(samples):
        predictions.append(monte_carlo_predictions(model, test_loader))
        
    predictions = np.stack(predictions , 0)
    mean_predictions = np.mean(predictions, axis=0)
    entropy = -1.0  * np.sum(mean_predictions * np.log(mean_predictions + 1e-16), axis=-1)
    return predictions, entropy

## Load Models
We will compare the results obtained from Monte Carlo Dropout with those from 10 instances of the OriginalSizeCNN model, each trained independently with different initial weights. All models were trained on the same data, providing a basis for evaluating the uncertainty in predictions across different training runs.

In [ ]:
def load_model(model, name) :
    model_path = f"./models/{name}.pth"
    model.load_state_dict(torch.load(model_path, weights_only=True,map_location=torch.device(device)))
    model.device = device
    model.to(device)

def load_dropout_model(model : src.model.DropoutCNN, name, p_cnl = 0.5, p_fl = 0.15):
    load_model(model,name)
    model.dropoutCNL.p = p_cnl
    model.dropoutFL.p = p_fl

In [ ]:
from src.dataset import prepare_dataset_loaders
from src.config import DATASET_DIR 
mean, std = load_mean_std(f"{DATASET_DIR}/scaling_params.json")

dropout_model = src.model.DropoutCNN() 
load_model(dropout_model, "DropoutCNN")

dropout_models = []
dropout_parameters = [(0.5, 0.15), (0.7, 0.3), (0.8,0.5)]
for (cnl_p, fl_p) in dropout_parameters:
    dropout_model = src.model.DropoutCNN() 
    load_dropout_model(dropout_model,"DropoutCNN",  cnl_p, fl_p)
    dropout_models.append(dropout_model)

ensemble_models = []

for i in range(10):
    sample_model = src.model.OriginalSizeCNN()
    model_name = f"Ensemble/OriginalSizeCNN_{i}"
    load_model(sample_model, model_name)
    ensemble_models.append(sample_model)

## 4. Get predictions from models


In [ ]:
batch_size = 10 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

mc_dropout_preditctions = []
mc_dropout_entropies = []
mc_dropout_variances = []

train_loader, val_loader, test_loader = prepare_dataset_loaders(transform, batch_size)

for model in dropout_models:
    predictions, entropy = monte_carlo_dropout(model, test_loader, samples=15)
    mc_dropout_entropies.append(entropy)
    mc_dropout_preditctions.append(predictions)
    mc_dropout_variances.append(np.var(predictions, axis=0))

ensemble_predictions = []
for model in ensemble_models:
    preds, _ = model_validate(model, test_loader)
    ensemble_predictions.append(preds)


## 5. Measures of Uncertainty
We will measure the uncertainty of the model by calculating the variance and entropy of predictions both for MC dropout and Ensemble of models. 

In [ ]:
import matplotlib.pyplot as plt

def calculate_entropy(predictions):
    -1.0  * np.sum(np.mean(predictions,axis=0) * np.log(np.mean(predictions,axis=0) + 1e-16), axis=-1);

ensemble_variances = np.var(ensemble_predictions, axis=0)
ensemble_entropy = calculate_entropy(ensemble_predictions)

for i in range(len(dropout_models)):
    print(f"Statistics of MC dropout model with dropout on CNL = {dropout_parameters[i][0]}  FL = {dropout_parameters[i][1]}")
    print("Mean variance ", np.mean(mc_dropout_variances[i]))
    print("Entropy: ", mc_dropout_entropies[i])

print("Mean variance of Ensemble model:", np.mean(ensemble_variances))
print("Ensemble Entropy:", ensemble_entropy)

The Monte Carlo (MC) Dropout models with higher dropout rates (CNL and FL) show increasing variance and entropy, indicating higher model uncertainty. Specifically, the model with a dropout rate of 0.8 on CNL and 0.5 on FL has the highest variance (0.196) and entropy (3640.08). In comparison, the Ensemble model exhibits a much lower variance of 0.0303, suggesting it is more stable and less uncertain. The higher variance and entropy in the MC dropout models imply that they explore different regions of the solution space, while the Ensemble model provides more reliable predictions with less uncertainty. It is 

## 6. Compare results of Monte Carlo dropout to Ensemble of models
We wanted to compare corelation of models sampled by MC dropout to the ensemble of models. To do so we present pearson matrix.
The results show that the average correlation is on the level of 68%, so the results vary.

In [ ]:
from scipy.stats import pearsonr
import numpy as np

dropout_flattened = [pred.flatten() for pred in mc_dropout_preditctions[0]]  
ensemble_flattened = [np.array(pred).flatten() for pred in ensemble_predictions] 
comparison_matrix = np.zeros((10, 10))

for i in range(10): 
    for j in range(10):  
        correlation, _ = pearsonr(dropout_flattened[i], ensemble_flattened[j])
        comparison_matrix[i, j] = correlation  

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(comparison_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Monte Carlo Dropout vs Ensemble Predictions")
plt.xlabel("Ensemble Model Index")
plt.ylabel("Dropout Pass Index")
plt.show()